<a href="https://colab.research.google.com/github/Aurelien07/stackoverflowquestions/blob/main/Projet_5_Analyse_supervis%C3%A9e.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import des libraires :

In [ ]:
!pip install transformers

In [ ]:
import pickle
import pandas as pd
import numpy as np
import IPython.display
import re

# Permettra de créer des accés avec google drive
import os

# Pour l'optimisation des algos :
from sklearn.model_selection import GridSearchCV

# pour les algorithmes supervisés :
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier


# Pour le BOW :
from nltk.tokenize import word_tokenize

# Pour le tf-idf :
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import tensorflow_hub as tf_hub

# Pour word2vec :
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import metrics as kmetrics
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import gensim
from gensim.models import Word2Vec, word2vec

# Pour la visualisation des tokens :
from sklearn.feature_extraction.text import CountVectorizer
from yellowbrick.text import FreqDistVisualizer

# Pour les scores :
from sklearn import metrics
from sklearn.metrics import accuracy_score, jaccard_score, hamming_loss
from sklearn.metrics import roc_auc_score

# Pour supprimer les warnings :
import warnings
warnings.filterwarnings("ignore")

### import des datas :

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df_modellisation = pd.read_pickle("/content/drive/My Drive/Colab Notebooks/TEST/df_modellisation_part_1.pkl")

In [ ]:
df_modellisation

### Mise en place de la limite des 50 tags :

In [ ]:
top_50_tags = pickle.load(open('/content/drive/My Drive/Colab Notebooks/TEST/top_50_tags.pkl', 'rb')) # pour le load

### Fonction pour les scores :

In [ ]:
def dico_metric (dico, col) :

  """
  source : https://scikit-learn.org/stable/modules/model_evaluation.html
  Permet de créer un dictionnaire avec les scores des differents algorithmes.
  """
  dico[col]  = {'Accuracy': round(metrics.accuracy_score(y_test, y_pred), 2) ,
                'Hamming loss' : round(metrics.hamming_loss(y_test, y_pred), 2),
              #  'Jaccard_score_macro' : round(metrics.jaccard_score(y_test, y_pred, average="macro"), 2),
                'Jaccard_score_micro' : round(metrics.jaccard_score(y_test, y_pred, average="micro"), 2),
              #  'f1_macro_score' : round(metrics.f1_score(y_test, y_pred, average='macro'), 2),
                'f1_micro_score' : round(metrics.f1_score(y_test, y_pred, average='micro'), 2),
                'Recall_micro_score' : round(metrics.f1_score(y_test, y_pred, average='micro'), 2),
              #  'Recall_macro_score' : round(metrics.f1_score(y_test, y_pred, average='macro'), 2),
                }
  return dico

# __Algorithme utilisant le BOW comme X :__

### Utiliser le bow en pickle :

In [ ]:
bow = pickle.load(open('/content/drive/My Drive/Colab Notebooks/TEST/bow.pkl', 'rb')) # pour le load

### Train test via BOW :

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix

In [ ]:
y = df_modellisation['tags_final']
multi_lab = MultiLabelBinarizer(classes= top_50_tags)
Y = multi_lab.fit_transform(y)

print("Affichage des classes du multilabel :")
display(multi_lab.classes_)

In [ ]:
X = bow # a modifier par tfidf et par wordtovec
y = Y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Création d'une liste pour les scores des métrics :

In [ ]:
model_metrics_bow = {} # Garde en mémoire les scores

## Regression logistique :

In [ ]:
def regression_log (X_train, y_train, X_test) :
  
  model_svm = OneVsRestClassifier(LogisticRegression())
  model_svm.fit(X_train, y_train)
  y_pred = model_svm.predict(X_test)
  
  return y_pred

In [ ]:
%%time
y_pred = regression_log (X_train, y_train, X_test)
#multiscore (y_test, y_pred)
dico_metric (model_metrics_bow, "reg_log")

## Random Forest :

In [ ]:
def random_forest (X_train, y_train, X_test) :

  model_rf = OneVsRestClassifier(RandomForestClassifier())
  model_rf.fit(X_train, y_train)
  y_pred = model_rf.predict(X_test) 
  
  return y_pred

In [ ]:
%%time
y_pred = random_forest (X_train, y_train, X_test)
#multiscore (y_test, y_pred)
dico_metric (model_metrics_bow, "random_forest")

## Decision tree : 

In [ ]:
def decision_tree (X_train, y_train, X_test) :

  model_tree = OneVsRestClassifier(DecisionTreeClassifier())
  model_tree.fit(X_train, y_train)
  y_pred = model_tree.predict(X_test) 
  
  return y_pred 

In [ ]:
%%time
y_pred = decision_tree (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_bow, "decision_tree")

## KNN : 

In [ ]:
def KNeighbors (X_train, y_train, X_test) :

  model_knn = OneVsRestClassifier(KNeighborsClassifier())
  model_knn.fit(X_train, y_train)
  y_pred = model_knn.predict(X_test)
  
  return y_pred

In [ ]:
%%time
y_pred = KNeighbors (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_bow, "knn")

## Xgboost :

In [ ]:
def xgboost (X_train, y_train, X_test) :

  model_xgb = OneVsRestClassifier(XGBRegressor())
  model_xgb.fit(X_train, y_train)
  y_pred = model_xgb.predict(X_test)  
  return y_pred

In [ ]:
%%time
y_pred = xgboost (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_bow, "xgboost")

## Gradient Boosting :

In [ ]:
def gradient_boost (X_train, y_train, X_test) :
  
  model_svm = OneVsRestClassifier(GradientBoostingClassifier())
  model_svm.fit(X_train, y_train)
  y_pred = model_svm.predict(X_test)
  
  return y_pred

In [ ]:
%%time
y_pred = gradient_boost (X_train, y_train, X_test)
# model_svc = multiscore (y_test, y_pred)
dico_metric (model_metrics_bow, "gradient_boost")

## AdaBoostClassifier :

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
def Ada_Boost_Classifier (X_train, y_train, X_test) :
  
  model_adab = OneVsRestClassifier(AdaBoostClassifier())
  model_adab.fit(X_train, y_train)
  y_pred = model_adab.predict(X_test)
  
  return y_pred

In [ ]:
%%time
y_pred = Ada_Boost_Classifier (X_train, y_train, X_test)
# model_svc = multiscore (y_test, y_pred)
dico_metric (model_metrics_bow, "Ada_Boost_Classifier")

### Dictionnaire des scores :

In [ ]:
score_bow = pd.DataFrame.from_dict(model_metrics_bow)
# score_bow = score_bow.style.applymap(color_yellow).applymap(make_bold)
print("-"*83)
print("-"*35 + f"Via le BOW :" + "-"*36)
print("-"*83)
display(score_bow)

#### Sauvegarde du résultat :

In [ ]:
#score_bow.to_csv("resultat_score_bow.csv", index=True) 
score_bow.to_csv("/content/drive/My Drive/Colab Notebooks/TEST/Resultat_score_bow.csv", index=True) 
# Cela permettra d'éviter d'attendre de relancer le notebook.

In [ ]:
score_bow = pd.read_csv("/content/drive/My Drive/Colab Notebooks/TEST/Resultat_score_bow.csv")
score_bow 

# __Algorithme utilisant le tf-idf comme X :__

Ayant déjà crée un TF-IDF sur le notebook précédent, on va donc réutiliser notre TF-IDF pour l'utiliser pour notre algorithme.

### Utiliser le tf-idf en pickle :

In [ ]:
idf = pickle.load(open('idf.pkl', 'rb')) # pour le load

### train test via Tf-IDF :

In [ ]:
X = idf # a modifier par tfidf et par wordtovec
y = Y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Création d'une liste pour les scores des métrics :

In [ ]:
model_metrics_idf = {} # Garde en mémoire les scores

## Regression logistique :

In [ ]:
%%time
y_pred = regression_log (X_train, y_train, X_test)
#multiscore (y_test, y_pred)
dico_metric (model_metrics_idf, "reg_log")

## Random Forest :

In [ ]:
%%time
y_pred = random_forest (X_train, y_train, X_test)
#multiscore (y_test, y_pred)
dico_metric (model_metrics_idf, "random_forest")

## Decision tree : 

In [ ]:
%%time
y_pred = decision_tree (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_idf, "decision_tree")

## KNN : 

In [ ]:
%%time
y_pred = KNeighbors (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_idf, "knn")

## Xgboost :

In [ ]:
%%time
y_pred = xgboost (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_idf, "xgboost")

## Gradient Boosting :

In [ ]:
%%time
y_pred = gradient_boost (X_train, y_train, X_test)
# model_svc = multiscore (y_test, y_pred)
dico_metric (model_metrics_idf, "gradient_boost")

## AdaBoostClassifier :

In [ ]:
%%time
y_pred = Ada_Boost_Classifier (X_train, y_train, X_test)
# model_svc = multiscore (y_test, y_pred)
dico_metric (model_metrics_idf, "Ada_Boost_Classifier")

### Dictionnaire des scores :

In [ ]:
score_tfidf = pd.DataFrame.from_dict(model_metrics_idf)

print("-"*83)
print("-"*34 + f"Via le TFIDF :" + "-"*35)
print("-"*83)
display(score_tfidf)

#### Sauvegarde du résultat :

In [ ]:
score_tfidf.to_csv("resultat_score_tfidf.csv", index=True) 

# __Algorithme utilisant le word2vec :__

In [ ]:
from nltk.corpus import stopwords
import string  # permet d'avoir accés à toute les ponctuations.

In [ ]:
def tokenize(text):

    stop_words = set(stopwords.words('english'))
    punctuation = set(string.punctuation)

    try:
        res = word_tokenize(text, language='english')
    except TypeError:
        return text

    res = [token for token in res if token not in punctuation]
    res = [token for token in res if token not in stop_words]
   # res = [token for token in res if token not in letters]
    return res

### Connaître le nombre de mots max :

In [ ]:
# Pour connaitre le max len :
df_modellisation['length_corpus'] = df_modellisation['corpus_finish'].apply(lambda x : len(tokenize(x)))
print(f"Dans le corpus, le nombre de mots maximum est de : ", df_modellisation['length_corpus'].max())

### Train test via word2vec :

In [ ]:
corpus = df_modellisation['corpus_finish']
corpus_count = 734

In [ ]:
!pip install gensim==4.2.0 # -> pour relancer le modéle 

### Mise en place de Word2vec :

In [ ]:
w2v_size=300
w2v_window=5
w2v_min_count=25
w2v_epochs=100
maxlen = 734 # taille des phrases à vérifier pour le max
sentences = df_modellisation['corpus_finish'].tolist()
sentences = [gensim.utils.simple_preprocess(text) for text in sentences]

### Entrainement du Word2vec :

In [ ]:
%%time
# Création et entraînement du modèle Word2Vec

print("Build & train Word2Vec model ...")
w2v_model = gensim.models.Word2Vec(min_count=w2v_min_count, window=w2v_window,
                                                vector_size=w2v_size,
                                                seed=42,
                                                workers=1)
#                                                workers=multiprocessing.cpu_count())
w2v_model.build_vocab(sentences)
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=w2v_epochs)
model_vectors = w2v_model.wv
w2v_words = model_vectors.index_to_key
print("Vocabulary size: %i" % len(w2v_words))
print("Word2Vec trained")

#### pickle du modéle :

In [ ]:
pickle.dump(w2v_model, open('w2v_model.pkl', 'wb'))

### Tokennisation du modéle :

In [ ]:
%%time
# Préparation des sentences (tokenization)

print("Fit Tokenizer ...")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
x_sentences = pad_sequences(tokenizer.texts_to_sequences(sentences),
                                                     maxlen=maxlen,
                                                     padding='post') 
                                                   
num_words = len(tokenizer.word_index) + 1
print("Number of unique words: %i" % num_words)

### Création de la matrice d'embedding :

In [ ]:
# Création de la matrice d'embedding

print("Create Embedding matrix ...")
w2v_size = 300
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, w2v_size))
i=0
j=0
    
for word, idx in word_index.items():
    i +=1
    if word in w2v_words:
        j +=1
        embedding_vector = model_vectors[word]
        if embedding_vector is not None:
            embedding_matrix[idx] = model_vectors[word]
            
word_rate = np.round(j/i,4)
print("Word embedding rate : ", word_rate)
print("Embedding matrix: %s" % str(embedding_matrix.shape))

### Création du modéle :

In [ ]:
# Création du modèle

input=Input(shape=(len(x_sentences),maxlen),dtype='float64')
word_input=Input(shape=(maxlen,),dtype='float64')  
word_embedding=Embedding(input_dim=vocab_size,
                         output_dim=w2v_size,
                         weights = [embedding_matrix],
                         input_length=maxlen)(word_input)
word_vec=GlobalAveragePooling1D()(word_embedding)  
embed_model = Model([word_input],word_vec)

embed_model.summary()

#### Pickle de  x_sentences :

In [ ]:
pickle.dump(x_sentences, open('x_sentences.pkl', 'wb'))
#x_sentences = pickle.load(open('x_sentences.pkl', 'rb')) # pour le load

### Train test via Word2vec :

In [ ]:
X = x_sentences # a modifier par tfidf et par wordtovec
y = Y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Création d'une liste pour les scores des métrics :

In [ ]:
model_metrics_word2vec = {} # Garde en mémoire les scores

## Regression Logistique :

In [ ]:
%%time
y_pred = regression_log (X_train, y_train, X_test)
#multiscore (y_test, y_pred)
dico_metric (model_metrics_word2vec, "reg_log")

## Random Forest :

In [ ]:
%%time
y_pred = random_forest (X_train, y_train, X_test)
#multiscore (y_test, y_pred)
dico_metric (model_metrics_word2vec, "random_forest")

## Decision tree : 

In [ ]:
%%time
y_pred = decision_tree (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_word2vec, "decision_tree")

## KNN : 

In [ ]:
%%time
y_pred = KNeighbors (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_word2vec, "knn")

## Xgboost :

In [ ]:
%%time
y_pred = xgboost (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_word2vec, "xgboost")

## Gradient Boosting :

In [ ]:
%%time
y_pred = gradient_boost (X_train, y_train, X_test)
# model_svc = multiscore (y_test, y_pred)
dico_metric (model_metrics_word2vec, "gradient_boost")

## AdaBoostClassifier :

In [ ]:
%%time
y_pred = Ada_Boost_Classifier (X_train, y_train, X_test)
# model_svc = multiscore (y_test, y_pred)
dico_metric (model_metrics_word2vec, "Ada_Boost_Classifier")

### Dictionnaire des scores :

In [ ]:
score_word2vec = pd.DataFrame.from_dict(model_metrics_word2vec)

print("-"*83)
print("-"*32 + f"Via le word2vec :" + "-"*34)
print("-"*83)
display(score_word2vec)

#### Sauvegarde du résultat :

In [ ]:
score_word2vec.to_csv("resultat_score_word2vec.csv") # Cela permettra d'éviter d'attendre de relancer le notebook.

# __Algorithme utilisant le BERT :__

### Fonction pour le BERT :

In [ ]:
# Fonction de préparation des sentences
def bert_inp_fct(sentences, bert_tokenizer, max_length) :
    input_ids=[]
    token_type_ids = []
    attention_mask=[]
    bert_inp_tot = []

    for sent in sentences:
        bert_inp = bert_tokenizer.encode_plus(sent,
                                              add_special_tokens = True,
                                              max_length = max_length,
                                              padding='max_length',
                                              return_attention_mask = True, 
                                              return_token_type_ids=True,
                                              truncation=True,
                                              return_tensors="tf")
    
        input_ids.append(bert_inp['input_ids'][0])
        token_type_ids.append(bert_inp['token_type_ids'][0])
        attention_mask.append(bert_inp['attention_mask'][0])
        bert_inp_tot.append((bert_inp['input_ids'][0], 
                             bert_inp['token_type_ids'][0], 
                             bert_inp['attention_mask'][0]))

    input_ids = np.asarray(input_ids)
    token_type_ids = np.asarray(token_type_ids)
    attention_mask = np.array(attention_mask)
    
    return input_ids, token_type_ids, attention_mask, bert_inp_tot


In [ ]:
# Fonction de création des features
def feature_BERT_fct(model, model_type, sentences, max_length, b_size, mode='HF') :
    batch_size = b_size
    batch_size_pred = b_size
    bert_tokenizer = AutoTokenizer.from_pretrained(model_type)
    time1 = time.time()

    for step in range(len(sentences)//batch_size) :
        idx = step*batch_size
        input_ids, token_type_ids, attention_mask, bert_inp_tot = bert_inp_fct(sentences[idx:idx+batch_size], 
                                                                      bert_tokenizer, max_length)
        
        if mode=='HF' :    # Bert HuggingFace
            outputs = model.predict([input_ids, attention_mask, token_type_ids], batch_size=batch_size_pred)
            last_hidden_states = outputs.last_hidden_state

        if mode=='TFhub' : # Bert Tensorflow Hub
            text_preprocessed = {"input_word_ids" : input_ids, 
                                 "input_mask" : attention_mask, 
                                 "input_type_ids" : token_type_ids}
            outputs = model(text_preprocessed)
            last_hidden_states = outputs['sequence_output']
             
        if step ==0 :
            last_hidden_states_tot = last_hidden_states
            last_hidden_states_tot_0 = last_hidden_states
        else :
            last_hidden_states_tot = np.concatenate((last_hidden_states_tot,last_hidden_states))
    
    features_bert = np.array(last_hidden_states_tot).mean(axis=1)
    
    time2 = np.round(time.time() - time1,0)
    print("temps traitement : ", time2)
     
    return features_bert, last_hidden_states_tot

### Mise en place du BERT :

In [ ]:
pip install -U tensorflow-text==2.10.0

In [ ]:
import tensorflow as tf
# import tensorflow_hub as hub
import tensorflow.keras
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import metrics as kmetrics
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

# Bert
import os
import transformers
from transformers import *

os.environ["TF_KERAS"]='1'

In [ ]:
print(tf .__version__)
print(tensorflow.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.test.is_built_with_cuda())

In [ ]:
max_length = 734
batch_size = 30
model_type = 'bert-base-uncased'
model = TFAutoModel.from_pretrained(model_type)
sentences = df_modellisation['corpus_finish'].tolist()

In [ ]:
import time
# Création des features

features_bert, last_hidden_states_tot = feature_BERT_fct(model, model_type, sentences, 
                                                         max_length, batch_size, mode='HF')

#### pickle de features_bert :

In [ ]:
pickle.dump(features_bert, open('features_bert.pkl', 'wb'))

### Train test via Bert :

In [ ]:
X = features_bert
y = Y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Création d'une liste pour les scores des métrics :

In [ ]:
model_metrics_bert = {} # Garde en mémoire les scores

## Regression Logistique :

In [ ]:
%%time
y_pred = regression_log (X_train, y_train, X_test)
#multiscore (y_test, y_pred)
dico_metric (model_metrics_bert, "reg_log")

## Random Forest :

In [ ]:
%%time
y_pred = random_forest (X_train, y_train, X_test)
#multiscore (y_test, y_pred)
dico_metric (model_metrics_bert, "random_forest")

## Decision Tree Classifier :

In [ ]:
%%time
y_pred = decision_tree (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_bert, "decision_tree")

## KNN : 

In [ ]:
%%time
y_pred = KNeighbors (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_bert, "knn")

## Xgboost :

In [ ]:
%%time
y_pred = xgboost (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_bert, "xgboost")

## Gradient Boosting :

In [ ]:
%%time
y_pred = gradient_boost (X_train, y_train, X_test)
# model_svc = multiscore (y_test, y_pred)
dico_metric (model_metrics_bert, "gradient_boost")

## AdaBoostClassifier :

In [ ]:
%%time
y_pred = Ada_Boost_Classifier (X_train, y_train, X_test)
# model_svc = multiscore (y_test, y_pred)
dico_metric (model_metrics_bert, "Ada_Boost_Classifier")

### Dictionnaire des scores :

In [ ]:
score_bert = pd.DataFrame.from_dict(model_metrics_bert)

print("-"*100)
print("-"*42 + f"Via le BERT :" + "-"*42)
print("-"*100)
display(score_bert)

#### Sauvegarde du résultat :

In [ ]:
score_bert.to_csv("resultat_score_bert.csv") # Cela permettra d'éviter d'attendre de relancer le notebook.

# __Algorithme utilisant USE :__

### Mise en place du USE :

In [ ]:
import tensorflow as tf
# import tensorflow_hub as hub
import tensorflow.keras
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import metrics as kmetrics
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import tensorflow_hub as hub

In [ ]:
%%time
module_url = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
model =  tf_hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

#### Mise au format array pour le X :

In [ ]:
%%time
embedding_sentence = embed(df_modellisation['corpus_finish'].tolist())
embedding_sentence = np.array(embedding_sentence)

#### pickle de embedding_sentence :

In [ ]:
# pickle.dump(embedding_sentence, open('embedding_sentence.pkl', 'wb'))
embedding_sentence = pickle.load(open('embedding_sentence.pkl', 'rb')) # pour le load

### Train test via USE :

In [ ]:
X = embedding_sentence # pour la méthode USE
y = Y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Création d'une liste pour les scores des métrics :

In [ ]:
model_metrics_use = {}

## Regression logistique :

In [ ]:
%%time
y_pred = regression_log (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_use, "reg_log")

## Random Forest :

In [ ]:
%%time
y_pred = random_forest (X_train, y_train, X_test)
#multiscore (y_test, y_pred)
dico_metric (model_metrics_use, "random_forest")

## Decision tree : 

In [ ]:
%%time
y_pred = decision_tree (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_use, "decision_tree")

## KNN : 

In [ ]:
%%time
y_pred = KNeighbors (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_use, "knn")

## Xgboost :

In [ ]:
%%time
y_pred = xgboost (X_train, y_train, X_test)
# multiscore (y_test, y_pred)
dico_metric (model_metrics_use, "xgboost")

## Gradient Boosting :

In [ ]:
%%time
y_pred = gradient_boost (X_train, y_train, X_test)
# model_svc = multiscore (y_test, y_pred)
dico_metric (model_metrics_use, "gradient_boost")

## AdaBoostClassifier :

In [ ]:
%%time
y_pred = Ada_Boost_Classifier (X_train, y_train, X_test)
# model_svc = multiscore (y_test, y_pred)
dico_metric (model_metrics_use, "Ada_Boost_Classifier")

### Dictionnaire des scores :

In [ ]:
score_use = pd.DataFrame.from_dict(model_metrics_use)

print("-"*100)
print("-"*42 + f"Via le use :" + "-"*42)
print("-"*100)
display(score_use)

#### Sauvegarde du résultat :

In [ ]:
score_bert.to_csv("resultat_score_bert.csv") # Cela permettra d'éviter d'attendre de relancer le notebook.

# Choix de l'algorithme supervisé :

## Visualisation des scores de toute les algorithmes utilisant des matrices d'embedding :